In [ ]:
import requests
import pandas as pd
from typing import Dict
import time

In [ ]:
# API call to sublevels of ministries
def fetch_budget_data(year: int, budget_id: str) -> Dict:
    url = "https://bundeshaushalt.de/internalapi/budgetData"
    params = {
        'year': year,
        'account': 'expenses',
        'quota': 'target',
        'unit': 'single',
        'id': budget_id
    }

    try:
        response = requests.get(url, params=params, timeout=10)

        if response.status_code == 200:
            data = response.json()
            return data
        else:
            print(f"     Error: Status Code {response.status_code}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"     Error fetching data: {e}")
        return None

def extract_detail_data(data: Dict, year: int, category_name: str, category_id: str) -> Dict:
    if not data or 'detail' not in data:
        return None

    detail = data['detail']
    row = {
        'Jahr': year,
        'ID': category_id,
        'Budgetnummer': detail.get('budgetNumber', ''),
        'Kategorie': category_name,
        'Wert_Euro': detail.get('value', 0),
        'Relativer_Wert_Prozent': detail.get('relativeValue', 0),
        'Relativer_Wert_zu_Eltern_Prozent': detail.get('relativeToParentValue', 0)
    }

    return row

def main():
    # Only subsidies to pension insurance subcategory
    subcategory_id = '1102'
    subcategory_name = 'Subsidies Pension Insurance'

    years = range(2014, 2027)  # 2014 to 2026
    all_data = []


    # Fetch data for each year
    for year in years:
        print(f"\nYear {year}:")
        print(f"  → Fetching: {subcategory_name} (ID: {subcategory_id})")

        data = fetch_budget_data(year, subcategory_id)

        if data:
            row = extract_detail_data(data, year, subcategory_name, subcategory_id)
            if row:
                all_data.append(row)
                print(f"     ✓ {row['Wert_Euro']:,.2f} €")
            else:
                print(f"     ✗ No data extracted from response")
        else:
            print(f"     ✗ Empty response returned")

        time.sleep(0.2)  # Short pause between requests

    # Create DataFrame
    df = pd.DataFrame(all_data)

    # Check if we have data
    if df.empty:
        print("\nNo data retrieved. Please check if the API is accessible.")
        return

    # Sort by year
    df = df.sort_values(['Jahr'])

    # Export to CSV
    output_file = 'project_chart4_pension_insurance.csv'
    df.to_csv(output_file, index=False)

    print("\n" + "=" * 60)
    print(f"✓ Success! Data saved to '{output_file}'.")
    print(f"  Total of {len(df)} records exported.")

    # Show summary
    print(f"\nTotal per year:")
    print(df.groupby('Jahr')['Wert_Euro'].sum().apply(lambda x: f"{x:,.0f} €"))

    print(f"\nAverage: {df['Wert_Euro'].mean():,.0f} € per year")

if __name__ == "__main__":
    main()


Year 2014:
  → Fetching: Subsidies Pension Insurance (ID: 1102)
     ✓ 88,428,480,000.00 €

Year 2015:
  → Fetching: Subsidies Pension Insurance (ID: 1102)
     ✓ 90,366,262,000.00 €

Year 2016:
  → Fetching: Subsidies Pension Insurance (ID: 1102)
     ✓ 93,271,172,000.00 €

Year 2017:
  → Fetching: Subsidies Pension Insurance (ID: 1102)
     ✓ 98,252,249,000.00 €

Year 2018:
  → Fetching: Subsidies Pension Insurance (ID: 1102)
     ✓ 100,142,526,000.00 €

Year 2019:
  → Fetching: Subsidies Pension Insurance (ID: 1102)
     ✓ 105,328,949,000.00 €

Year 2020:
  → Fetching: Subsidies Pension Insurance (ID: 1102)
     ✓ 109,901,960,000.00 €

Year 2021:
  → Fetching: Subsidies Pension Insurance (ID: 1102)
     ✓ 114,669,405,000.00 €

Year 2022:
  → Fetching: Subsidies Pension Insurance (ID: 1102)
     ✓ 116,785,160,000.00 €

Year 2023:
  → Fetching: Subsidies Pension Insurance (ID: 1102)
     ✓ 121,049,523,000.00 €

Year 2024:
  → Fetching: Subsidies Pension Insurance (ID: 1102)
     ✓ 12